<a href="https://colab.research.google.com/github/wajihakanwal/RAG-Hello-World/blob/main/RAG_Hello_World.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai langchain-pinecone

In [ ]:
!pip install -qU pypdf

In [ ]:
!pip install -Uq langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata

userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello World")

In [ ]:
from pickle import TRUE
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
from typing import List

index_name = "quickstart4"

pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key= pinecone_api_key)


indices_list = []
indices_list = pc.list_indexes()

#print (f"List indices:  {indices_list}")
index_found = False

# Extract and print the "name" attributes
for index in indices_list.indexes:
    #print(f"Index Name: {index['name']}")
    if index_name == index['name']:
      index_found = True
      break


if index_found == False:
    pc.create_index(
    name=index_name,
    dimension=768, #Replace with your model dimensions, here we r using google's model
    metric="cosine",  #Replace with your model metric, we r using cosine search method in indexes
    spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )
else:
    print(f"Index '{index_name}' already exists. Skipping Creations")


  # If index exists, connect to the existing index instead of creating a new one
  # index = pc.Index(index_name) # This line is already present after the if-else block

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index,embedding=embeddings)


Index 'quickstart4' already exists. Skipping Creations


In [ ]:
import langchain.document_loaders
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from uuid import uuid5, NAMESPACE_DNS

# Load documents
#Each load returns a list that is saved in respective doc[i].
doc1 = TextLoader("/content/Document1.txt").load()
doc2 = TextLoader("/content/Document2.txt").load()
doc3 = TextLoader("/content/Document3.txt").load()
doc4 = TextLoader("/content/document0.txt").load()

#variable document now saves contenation of lists to be passed to vector_store
documents = doc1+doc2+doc3+doc4

"""
documents = [doc1,doc2,doc3,doc4] will make another list of lists that will give error
as the expected attribute is a list of documents, not the list of lists
"""
uuids = [str(uuid5(NAMESPACE_DNS, f"doc-{i}")) for i in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)
#print(f"Documents = {documents}")

#split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
#print(f"Split Docs = {docs}")


Documents = [Document(metadata={'source': '/content/Document1.txt', 'text': 'Ibn Sina: The Polymath of the Golden Age of Islam\n\nAbū ʿAlī al-Ḥusayn ibn ʿAbd Allāh ibn Sīnā, known in the West as Avicenna, is one of the most celebrated figures in the history of Islamic civilization. Born in 980 CE in Afshana near Bukhara (modern-day Uzbekistan), Ibn Sina became a towering intellectual force, contributing profoundly to various disciplines, including medicine, philosophy, astronomy, mathematics, and theology. His works have left an indelible impact on both Eastern and Western thought, earning him titles such as “The Father of Modern Medicine” and “The Prince of Physicians.” This article explores his life, achievements, and enduring legacy.\n\nEarly Life and Education\n\nIbn Sina was born during the Islamic Golden Age, a period marked by flourishing intellectual and cultural advancements. His father was an official in the Samanid court, which provided the young Ibn Sina access to an extens

In [ ]:
response = vector_store.similarity_search("What was the contribution of Ibne Haytham")
print(f"Sure: Your Query Response is: {response[0].page_content}")

while True:
  response = input("What do you want to know: (q-quit)? ")
  if response.lower() == "q":
    break
  response = vector_store.similarity_search(response)
  print(f"Answer: {response[0].page_content}")
